# ``2. Encoding``

In [1]:
# Lectura de datos
# ==============================================================================
import pickle

# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import sidetable
from sklearn.preprocessing import OneHotEncoder

# Estadísticos
# ==============================================================================
from scipy import stats
import researchpy as rp
from scipy.stats import levene

# Librerías para manejar fechas
# ==============================================================================
from datetime import datetime
import calendar

# Desplegar todas las columnas
# ==============================================================================
pd.options.display.max_columns = None

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

# Test de Shapiro
# ==============================================================================
from scipy import stats

# Asimetría
# ==============================================================================
from scipy.stats import skew

# Curtosis
# ==============================================================================
from scipy.stats import kurtosistest

# Test de Levene
# ==============================================================================
from scipy.stats import levene

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Tamaño gráficas
# ==============================================================================
plt.rcParams["figure.figsize"] = (10,8)

In [2]:
df = pd.read_pickle('datos/1.bikes_limpio.pkl')
#df = pd.read_csv('datos/1.bikes_limpio.csv', index_col=0).reset_index(drop=True)
df.head()

,dteday,seasons,yr,mnth,holiday_num,weekday_num,weekday_cat,workingday_num,weathersit,atemp,hum,windspeed,casual,registered
instant,,,,,,,,,,,,,,
1,2018-01-01,winter,0,1,1,0,monday,0,2,18.18125,80.5833,10.749882,331,654
2,2018-01-02,winter,0,1,0,1,tuesday,1,2,17.68695,69.6087,16.652113,131,670
3,2018-01-03,winter,0,1,0,2,wednesday,1,1,9.47025,43.7273,16.636703,120,1229
4,2018-01-04,winter,0,1,0,3,thursday,1,1,10.60610,59.0435,10.739832,108,1454
5,2018-01-05,winter,0,1,0,4,friday,1,1,11.46350,43.6957,12.522300,82,1518


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 1 to 730
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   dteday          730 non-null    datetime64[ns]
 1   seasons         730 non-null    category      
 2   yr              730 non-null    category      
 3   mnth            730 non-null    category      
 4   holiday_num     730 non-null    category      
 5   weekday_num     730 non-null    category      
 6   weekday_cat     730 non-null    category      
 7   workingday_num  730 non-null    category      
 8   weathersit      730 non-null    category      
 9   atemp           730 non-null    float64       
 10  hum             730 non-null    float64       
 11  windspeed       730 non-null    float64       
 12  casual          730 non-null    int64         
 13  registered      730 non-null    int64         
dtypes: category(8), datetime64[ns](1), float64(3), int64(2)
me

In [4]:
round(df.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
atemp,730.0,23.73,8.15,3.95,16.89,24.37,30.45,42.04
hum,730.0,62.91,13.95,25.42,52.22,62.65,72.99,97.25
windspeed,730.0,12.48,4.77,1.50,9.04,12.13,15.41,25.33
casual,730.0,732.02,505.23,2.00,316.25,715.00,1000.25,2258.00
registered,730.0,3658.76,1559.76,20.00,2502.25,3664.50,4783.25,6946.00


# 2.1 Encoding con OneHotEncoder ya que no le vamos a dar orden de importancia a las variables:

## 2.1.1. Selección de variables a codificar

In [5]:
df.columns

Index(['dteday', 'seasons', 'yr', 'mnth', 'holiday_num', 'weekday_num',
       'weekday_cat', 'workingday_num', 'weathersit', 'atemp', 'hum',
       'windspeed', 'casual', 'registered'],
      dtype='object')

In [6]:
lista_codificar = ['yr', 'mnth', 'holiday_num', 'weekday_num',
        'workingday_num', 'weathersit']

## 2.1.2 Función método One Hot Encoder

In [7]:
# vamos a definir una función que nos aplique este método

def one_hot_encoder_one(df,columna,keep_first=True):
    
    # iniciamos el método de OneHot Encoder
    oh = OneHotEncoder()
    
    # hacemos la codificación de los datos para la variable dada 
    transformados = oh.fit_transform(df[[columna]])
    
    # convertimos nuestro array con la codificación hecha en un dataframe
    oh_df = pd.DataFrame(transformados.toarray())
    
    # el método get_feature_names nos va a dar el nombre de las columnas nuevas que se nos generarán
    oh_df.columns = columna+oh.get_feature_names_out()
    
    # concatenamos el dataframe original con el dataframe que acabamos de crear
    final = pd.concat([df,oh_df],axis=1)
    
    # eliminamos la columna original 
    final.drop(columna, axis = 1,  inplace = True)
    return final

In [8]:
df1 = df.copy()
df1.head()

,dteday,seasons,yr,mnth,holiday_num,weekday_num,weekday_cat,workingday_num,weathersit,atemp,hum,windspeed,casual,registered
instant,,,,,,,,,,,,,,
1,2018-01-01,winter,0,1,1,0,monday,0,2,18.18125,80.5833,10.749882,331,654
2,2018-01-02,winter,0,1,0,1,tuesday,1,2,17.68695,69.6087,16.652113,131,670
3,2018-01-03,winter,0,1,0,2,wednesday,1,1,9.47025,43.7273,16.636703,120,1229
4,2018-01-04,winter,0,1,0,3,thursday,1,1,10.60610,59.0435,10.739832,108,1454
5,2018-01-05,winter,0,1,0,4,friday,1,1,11.46350,43.6957,12.522300,82,1518


In [9]:
df1.reset_index(inplace=True)

In [10]:
df1.head(2)

,instant,dteday,seasons,yr,mnth,holiday_num,weekday_num,weekday_cat,workingday_num,weathersit,atemp,hum,windspeed,casual,registered
0,1,2018-01-01,winter,0,1,1,0,monday,0,2,18.18125,80.5833,10.749882,331,654
1,2,2018-01-02,winter,0,1,0,1,tuesday,1,2,17.68695,69.6087,16.652113,131,670


In [11]:
df1.drop(["instant",'dteday','weekday_cat'], axis=1, inplace=True)

In [12]:
df1.isnull().sum()

seasons           0
yr                0
mnth              0
holiday_num       0
weekday_num       0
workingday_num    0
weathersit        0
atemp             0
hum               0
windspeed         0
casual            0
registered        0
dtype: int64

In [13]:
df1.head(2)

,seasons,yr,mnth,holiday_num,weekday_num,workingday_num,weathersit,atemp,hum,windspeed,casual,registered
0,winter,0,1,1,0,0,2,18.18125,80.5833,10.749882,331,654
1,winter,0,1,0,1,1,2,17.68695,69.6087,16.652113,131,670


## 2.2.3 Aplicar método oh

In [14]:
lista_codificar

['yr', 'mnth', 'holiday_num', 'weekday_num', 'workingday_num', 'weathersit']

In [15]:
for i in lista_codificar:
    df1 = one_hot_encoder_one(df1, i)

In [16]:
df1.head(2)

,seasons,atemp,hum,windspeed,casual,registered,yryr_0,yryr_1,mnthmnth_1,mnthmnth_2,mnthmnth_3,mnthmnth_4,mnthmnth_5,mnthmnth_6,mnthmnth_7,mnthmnth_8,mnthmnth_9,mnthmnth_10,mnthmnth_11,mnthmnth_12,holiday_numholiday_num_0,holiday_numholiday_num_1,weekday_numweekday_num_0,weekday_numweekday_num_1,weekday_numweekday_num_2,weekday_numweekday_num_3,weekday_numweekday_num_4,weekday_numweekday_num_5,weekday_numweekday_num_6,workingday_numworkingday_num_0,workingday_numworkingday_num_1,weathersitweathersit_1,weathersitweathersit_2,weathersitweathersit_3
0,winter,18.18125,80.5833,10.749882,331,654,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,winter,17.68695,69.6087,16.652113,131,670,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [17]:
df1.isnull().sum()

seasons                           0
atemp                             0
hum                               0
windspeed                         0
casual                            0
registered                        0
yryr_0                            0
yryr_1                            0
mnthmnth_1                        0
mnthmnth_2                        0
mnthmnth_3                        0
mnthmnth_4                        0
mnthmnth_5                        0
mnthmnth_6                        0
mnthmnth_7                        0
mnthmnth_8                        0
mnthmnth_9                        0
mnthmnth_10                       0
mnthmnth_11                       0
mnthmnth_12                       0
holiday_numholiday_num_0          0
holiday_numholiday_num_1          0
weekday_numweekday_num_0          0
weekday_numweekday_num_1          0
weekday_numweekday_num_2          0
weekday_numweekday_num_3          0
weekday_numweekday_num_4          0
weekday_numweekday_num_5    

## 2.2 Map columna seasons

In [18]:
df1["seasons"].unique()

['winter', 'spring', 'summer', 'autumn']
Categories (4, object): ['autumn', 'spring', 'summer', 'winter']

In [19]:
mapa = {'winter':1, 'spring':2, 'summer':3 , 'autumn':2}

In [20]:
df1["seasons"] = df1["seasons"].map(mapa)

In [21]:
df1.sample(5)

,seasons,atemp,hum,windspeed,casual,registered,yryr_0,yryr_1,mnthmnth_1,mnthmnth_2,mnthmnth_3,mnthmnth_4,mnthmnth_5,mnthmnth_6,mnthmnth_7,mnthmnth_8,mnthmnth_9,mnthmnth_10,mnthmnth_11,mnthmnth_12,holiday_numholiday_num_0,holiday_numholiday_num_1,weekday_numweekday_num_0,weekday_numweekday_num_1,weekday_numweekday_num_2,weekday_numweekday_num_3,weekday_numweekday_num_4,weekday_numweekday_num_5,weekday_numweekday_num_6,workingday_numworkingday_num_0,workingday_numworkingday_num_1,weathersitweathersit_1,weathersitweathersit_2,weathersitweathersit_3
73,1,15.90890,65.5652,12.348703,289,1767,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
257,3,27.68355,70.9167,18.166782,428,3231,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
723,1,14.72325,73.4783,11.304642,440,573,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
111,2,16.09770,72.9583,14.707907,177,1506,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
99,2,21.33685,85.7500,9.833389,1188,1707,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [22]:
df1.isnull().sum()

seasons                           0
atemp                             0
hum                               0
windspeed                         0
casual                            0
registered                        0
yryr_0                            0
yryr_1                            0
mnthmnth_1                        0
mnthmnth_2                        0
mnthmnth_3                        0
mnthmnth_4                        0
mnthmnth_5                        0
mnthmnth_6                        0
mnthmnth_7                        0
mnthmnth_8                        0
mnthmnth_9                        0
mnthmnth_10                       0
mnthmnth_11                       0
mnthmnth_12                       0
holiday_numholiday_num_0          0
holiday_numholiday_num_1          0
weekday_numweekday_num_0          0
weekday_numweekday_num_1          0
weekday_numweekday_num_2          0
weekday_numweekday_num_3          0
weekday_numweekday_num_4          0
weekday_numweekday_num_5    

In [23]:
# Creamos los df con lo que se trabajará casuales
df_casual = df1.drop(['registered'], axis=1)

In [24]:
df_registered = df1.drop(['casual'], axis=1)

In [25]:
df1.to_csv('datos/2.encoding.csv')
df1.to_pickle('datos/2.encoding.pkl')

In [26]:
#guardamos en csv y en pkl los df con los datos de registrados:

df_registered.to_csv('datos/2.reg_encoding.csv')
df_registered.to_pickle('datos/2.reg_encoding.pkl')

In [27]:
#guardamos en csv y en pkl los df con los datos de registrados:

df_casual.to_csv('datos/2.cas_encoding.csv')
df_casual.to_pickle('datos/2.cas_encoding.pkl')